In [1]:
import numpy as np

In [8]:
np.random.seed(1)
x = np.random.randn(4, 3, 3, 2)
x.shape
x

array([[[[ 1.62434536, -0.61175641],
         [-0.52817175, -1.07296862],
         [ 0.86540763, -2.3015387 ]],

        [[ 1.74481176, -0.7612069 ],
         [ 0.3190391 , -0.24937038],
         [ 1.46210794, -2.06014071]],

        [[-0.3224172 , -0.38405435],
         [ 1.13376944, -1.09989127],
         [-0.17242821, -0.87785842]]],


       [[[ 0.04221375,  0.58281521],
         [-1.10061918,  1.14472371],
         [ 0.90159072,  0.50249434]],

        [[ 0.90085595, -0.68372786],
         [-0.12289023, -0.93576943],
         [-0.26788808,  0.53035547]],

        [[-0.69166075, -0.39675353],
         [-0.6871727 , -0.84520564],
         [-0.67124613, -0.0126646 ]]],


       [[[-1.11731035,  0.2344157 ],
         [ 1.65980218,  0.74204416],
         [-0.19183555, -0.88762896]],

        [[-0.74715829,  1.6924546 ],
         [ 0.05080775, -0.63699565],
         [ 0.19091548,  2.10025514]],

        [[ 0.12015895,  0.61720311],
         [ 0.30017032, -0.35224985],
         [-1.14251

In [24]:
y = np.zeros((3,2))
y
x = np.random.randn(2,3, 3, 2)

(2, 3, 3, 2)

In [34]:
np.random.seed(1)
A_prev = np.random.randn(2,4,4,3)
A_prev[0]
#A_prev.shape[0]

array([[[ 1.62434536, -0.61175641, -0.52817175],
        [-1.07296862,  0.86540763, -2.3015387 ],
        [ 1.74481176, -0.7612069 ,  0.3190391 ],
        [-0.24937038,  1.46210794, -2.06014071]],

       [[-0.3224172 , -0.38405435,  1.13376944],
        [-1.09989127, -0.17242821, -0.87785842],
        [ 0.04221375,  0.58281521, -1.10061918],
        [ 1.14472371,  0.90159072,  0.50249434]],

       [[ 0.90085595, -0.68372786, -0.12289023],
        [-0.93576943, -0.26788808,  0.53035547],
        [-0.69166075, -0.39675353, -0.6871727 ],
        [-0.84520564, -0.67124613, -0.0126646 ]],

       [[-1.11731035,  0.2344157 ,  1.65980218],
        [ 0.74204416, -0.19183555, -0.88762896],
        [-0.74715829,  1.6924546 ,  0.05080775],
        [-0.63699565,  0.19091548,  2.10025514]]])

In [33]:
A_prev.shape[0]

2

In [88]:
def zero_pad(X, pad):
    """
    Pad with zeros all images of the dataset X. The padding is applied to the height and width of an image, 
    as illustrated in Figure 1.
    
    Argument:
    X -- python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
    pad -- integer, amount of padding around each image on vertical and horizontal dimensions
    
    Returns:
    X_pad -- padded image of shape (m, n_H + 2*pad, n_W + 2*pad, n_C)
    """
    
    ### START CODE HERE ### (≈ 1 line)
    X_pad = np.pad(X, ((0,0), (pad, pad), (pad,pad), (0,0)), 'constant', constant_values=(0, 0))
    ### END CODE HERE ###
    
    return X_pad

# GRADED FUNCTION: conv_forward

def conv_forward(A_prev, W, b, hparameters):
    """
    Implements the forward propagation for a convolution function
    
    Arguments:
    A_prev -- output activations of the previous layer, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    W -- Weights, numpy array of shape (f, f, n_C_prev, n_C)
    b -- Biases, numpy array of shape (1, 1, 1, n_C)
    hparameters -- python dictionary containing "stride" and "pad"
        
    Returns:
    Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward() function
    """
    
    ### START CODE HERE ###
    # Retrieve dimensions from A_prev's shape (≈1 line)  
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Retrieve dimensions from W's shape (≈1 line)
    (f, f, n_C_prev, n_C) = W.shape
    
    # Retrieve information from "hparameters" (≈2 lines)
    stride = hparameters["stride"]
    pad = hparameters["pad"]
    
    # Compute the dimensions of the CONV output volume using the formula given above. Hint: use int() to floor. (≈2 lines)
    n_H = int(((n_H_prev - f + 2 * pad) / (stride)) + 1)
    n_W = int(((n_W_prev - f + 2 * pad) / (stride)) + 1)
    
    # Initialize the output volume Z with zeros. (≈1 line)
    Z = np.zeros((m, n_H_prev, n_W_prev, n_C_prev))
    
    # Create A_prev_pad by padding A_prev
    A_prev_pad = zero_pad(A_prev, pad)
    
    for i in range(m):                               # loop over the batch of training examples
        a_prev_pad = A_prev_pad[i,:,:,:]                   # Select ith training example's padded activation
        for h in range(n_H_prev):                           # loop over vertical axis of the output volume
            for w in range(n_W_prev):                       # loop over horizontal axis of the output volume
                for c in range(n_C_prev):                   # loop over channels (= #filters) of the output volume
                    
                    # Find the corners of the current "slice" (≈4 lines)
                    vert_start = h * stride
                    vert_end = h * stride + f
                    horiz_start = w * stride
                    horiz_end = w * stride + f
                    
                    # Use the corners to define the (3D) slice of a_prev_pad (See Hint above the cell). (≈1 line)
                    a_slice = A_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]
                    
                    # Convolve the (3D) slice with the correct filter W and bias b, to get back one output neuron. (≈1 line)
                    Z[i, h, w, c] = conv_single_step(a_slice, W[:,:,:,c], b[:,:,:,c])
                                        
    ### END CODE HERE ###
    
    # Making sure your output shape is correct
    assert(Z.shape == (m, n_H, n_W, n_C))
    
    # Save information in "cache" for the backprop
    cache = (A_prev, W, b, hparameters)
    
    return Z, cache

In [85]:
np.random.seed(1)
A_prev = np.random.randn(1,4,4,3)
W = np.random.randn(2,2,3,8)
b = np.random.randn(1,1,1,8)
hparameters = {"pad" : 2,
               "stride": 2}

#A_prev_pad = conv_forward(A_prev, W, b, hparameters)

In [86]:
A_prev.shape

(1, 4, 4, 3)

In [87]:
A_prev

array([[[[ 1.62434536, -0.61175641, -0.52817175],
         [-1.07296862,  0.86540763, -2.3015387 ],
         [ 1.74481176, -0.7612069 ,  0.3190391 ],
         [-0.24937038,  1.46210794, -2.06014071]],

        [[-0.3224172 , -0.38405435,  1.13376944],
         [-1.09989127, -0.17242821, -0.87785842],
         [ 0.04221375,  0.58281521, -1.10061918],
         [ 1.14472371,  0.90159072,  0.50249434]],

        [[ 0.90085595, -0.68372786, -0.12289023],
         [-0.93576943, -0.26788808,  0.53035547],
         [-0.69166075, -0.39675353, -0.6871727 ],
         [-0.84520564, -0.67124613, -0.0126646 ]],

        [[-1.11731035,  0.2344157 ,  1.65980218],
         [ 0.74204416, -0.19183555, -0.88762896],
         [-0.74715829,  1.6924546 ,  0.05080775],
         [-0.63699565,  0.19091548,  2.10025514]]]])

In [81]:
W.shape

(2, 2, 3, 8)

In [82]:

W


array([[[[ 0.5154138 , -1.11487105, -0.76730983,  0.67457071,  1.46089238,
           0.5924728 ,  1.19783084,  1.70459417],
         [ 1.04008915, -0.91844004, -0.10534471,  0.63019567, -0.4148469 ,
           0.45194604, -1.57915629, -0.82862798],
         [ 0.52887975, -2.23708651, -1.1077125 , -0.01771832, -1.71939447,
           0.057121  , -0.79954749, -0.2915946 ]],

        [[-0.25898285,  0.1892932 , -0.56378873,  0.08968641, -0.6011568 ,
           0.55607351,  1.69380911,  0.19686978],
         [ 0.16986926, -1.16400797,  0.69336623, -0.75806733, -0.8088472 ,
           0.55743945,  0.18103874,  1.10717545],
         [ 1.44287693, -0.53968156,  0.12837699,  1.76041518,  0.96653925,
           0.71304905,  1.30620607, -0.60460297]]],


       [[[ 0.63658341,  1.40925339,  1.62091229, -0.80618482, -0.25167421,
           0.38271517, -0.28899734, -0.39181624],
         [ 0.68400133, -0.35340998, -1.78791289,  0.36184732, -0.42449279,
          -0.73153098, -1.56573815,  1.01382